# source ; https://www.justintodata.com/pandas-groupby-with-python/

    - What is a Pandas 'GroupBy' (object)
    
    - How to create summary statistics for groups with 'aggregation' functions.
    
    - How to create like-indexed objects of statistics for groups with the 'transformation' method
    
    - How to use the flexible yet less efficient 'apply' function.
    
    - How to use 'custom functions' for 'multiple columns'

## What are pandas and GroupBy?

'Pandas' is a powerful and easy to use open-source Python data analysis and manipulation tool. It offers data structures and operations for numerical tables and time series. pandas can be used to import data, manipulate, and clean data.

It is a must-know package for data science.

'Group by'

    - Splitting the data into groups based on some criteria.
    
    - Applying a function to each group independently.
    
    - Combining the results into a data structure.
    
This can be used to group large amounts of data and compute operations on  these groups.
The idea should be familiar to those who have used SQL's SELECT and Group By statement.

We'll be going through the most common use cases for GroupBy in data science based on our experience. We recommend downloading the same dataset and practice along.

more detail is here 'https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html'

In [22]:
import pandas as pd
import numpy as np

from datetime import timedelta

    - converting the date of the transaction to datetime
    
    - translating the type of the transaction to English
    
    - translating the operation type of the transaction to English
    
    - filling in missing values of type and operation as 'Unknown' We'll be using these two variables as groups
    
    - sorting the DataFrame by account_id and date
    
    - setting the date variable as index

In [30]:
df = pd.read_csv('https://query.data.world/s/pvcoey3omglfxzdrfx6pyepphppqbl', delimiter=';', low_memory=False)

In [31]:
df.head(10)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN
5,771035,2632,930102,PRIJEM,VKLAD,1100.0,1100.0,NaN,NaN,NaN
6,452728,1539,930103,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
7,725751,2484,930103,PRIJEM,VKLAD,1100.0,1100.0,NaN,NaN,NaN
8,497211,1695,930103,PRIJEM,VKLAD,200.0,200.0,NaN,NaN,NaN
9,232960,793,930103,PRIJEM,VKLAD,800.0,800.0,NaN,NaN,NaN


In [32]:
# Convert the date into date format.
df['date'] = pd.to_datetime(df['date'], format='%y%m%d')

# Change the transaction types to English.
to_replace = {'PRIJEM': 'CREDIT', 'VYDAJ': 'WITHDRAWAL', 'VYBER': 'NOT SURE'}
df['type'] = df['type'].replace(to_replace).fillna('Unknown')

# Change the transaction operations to English.
to_replace = {'VYBER KARTOU': 'credit card withdrawal',
              'VKLAD': 'credit in cash',
              'PREVOD Z UCTU': 'collection from another bank',
              'VYBER': 'withdrawal in cash',
              'PREVOD NA UCET': 'remittance to another bank'}
df['operation'] = df['operation'].replace(to_replace).fillna('Unknown')

# sort the dataframe because it will be eaiser to see some things later.
df = df.sort_values(by=['account_id', 'date'])

# since we're going to do a time series example set the index to be the date.
df = df.set_index('date')

df.head(10)

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account
date,,,,,,,,,
1995-03-24,1,1,CREDIT,credit in cash,1000.0,1000.0,NaN,NaN,NaN
1995-04-13,5,1,CREDIT,collection from another bank,3679.0,4679.0,NaN,AB,41403269.0
1995-04-23,199,1,CREDIT,credit in cash,12600.0,17279.0,NaN,NaN,NaN
1995-04-30,3530438,1,CREDIT,Unknown,19.2,17298.2,UROK,NaN,NaN
1995-05-13,6,1,CREDIT,collection from another bank,3679.0,20977.2,NaN,AB,41403269.0
1995-05-23,200,1,CREDIT,credit in cash,2100.0,23077.2,NaN,NaN,NaN
1995-05-31,3530439,1,CREDIT,Unknown,79.0,23156.2,UROK,NaN,NaN
1995-06-13,7,1,CREDIT,collection from another bank,3679.0,26835.2,NaN,AB,41403269.0
1995-06-22,201,1,WITHDRAWAL,withdrawal in cash,200.0,26635.2,NaN,NaN,NaN


__Related article ; https://www.justintodata.com/unsupervised-anomaly-detection-on-bank-transactions-outliers/__

Within the previous article, the same dataset was used for anomaly detection.

## GroupBy Object; Splitting an Object into Groups

We are going to use variables type and operation to group the data.

In [33]:
df['type'].value_counts(dropna=False)

df['operation'].value_counts(dropna=False)

withdrawal in cash              434918
remittance to another bank      208283
Unknown                         183114
credit in cash                  156743
collection from another bank     65226
credit card withdrawal            8036
Name: operation, dtype: int64

To start the groupby process, we create a _GroupBy_ object called _grouped_. This helps in splitting the pandas objects into groups.

In [45]:
grouped = df.groupby(['type', 'operation'])

type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

By using the type function on grouped, we know that it is an object of pandas.core.goupby.generic.DataFrameGroupBy

The GroupBy object has methods we can call to manipulate each group.

For example, we can use the 'groups' method to get a dictionary with;

    - keys being the groups and
    
    - values being the indices of the rows within the groups, which is the date.

In [47]:
# This returns a dictionary with the keys being the groups and the values of the keys being the indices of the rows in the group.
grouped.groups

{('CREDIT', 'Unknown'): [1995-04-30 00:00:00, 1995-05-31 00:00:00, 1995-06-30 00:00:00, 1995-07-31 00:00:00, 1995-08-31 00:00:00, 1995-09-30 00:00:00, 1995-10-31 00:00:00, 1995-11-30 00:00:00, 1995-12-31 00:00:00, 1996-01-31 00:00:00, 1996-02-29 00:00:00, 1996-03-31 00:00:00, 1996-04-30 00:00:00, 1996-05-31 00:00:00, 1996-06-30 00:00:00, 1996-07-31 00:00:00, 1996-08-31 00:00:00, 1996-09-30 00:00:00, 1996-10-31 00:00:00, 1996-11-30 00:00:00, 1996-12-31 00:00:00, 1997-01-31 00:00:00, 1997-02-28 00:00:00, 1997-03-31 00:00:00, 1997-04-30 00:00:00, 1997-05-31 00:00:00, 1997-06-30 00:00:00, 1997-07-31 00:00:00, 1997-08-31 00:00:00, 1997-09-30 00:00:00, 1997-10-31 00:00:00, 1997-11-30 00:00:00, 1997-12-31 00:00:00, 1998-01-31 00:00:00, 1998-02-28 00:00:00, 1998-03-31 00:00:00, 1998-04-30 00:00:00, 1998-05-31 00:00:00, 1998-06-30 00:00:00, 1998-07-31 00:00:00, 1998-08-31 00:00:00, 1998-09-30 00:00:00, 1998-10-31 00:00:00, 1998-11-30 00:00:00, 1998-12-31 00:00:00, 1993-03-31 00:00:00, 1993-04-3

We can also get the DataFrame of a group using the 'get_group' method . For example, we can create a new dataset with only the 'credit card withdrawal'.

In [52]:
df_ccw = grouped.get_group(('WITHDRAWAL', 'credit card withdrawal'))

# This should be all (withdrawal, credit card withdrawal) transactions.
df_ccw.head(10)

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account
date,,,,,,,,,
1998-12-21,1761,7,WITHDRAWAL,credit card withdrawal,5900.0,68175.3,NaN,NaN,0.0
1998-05-29,3416,14,WITHDRAWAL,credit card withdrawal,1800.0,40773.1,NaN,NaN,0.0
1998-07-20,3474,14,WITHDRAWAL,credit card withdrawal,3000.0,38361.6,NaN,NaN,0.0
1998-08-19,3475,14,WITHDRAWAL,credit card withdrawal,2800.0,46829.7,NaN,NaN,0.0
1998-08-31,3417,14,WITHDRAWAL,credit card withdrawal,2800.0,44203.1,NaN,NaN,0.0
1998-11-22,3418,14,WITHDRAWAL,credit card withdrawal,1000.0,38156.6,NaN,NaN,0.0
1998-12-15,3408,14,WITHDRAWAL,credit card withdrawal,1600.0,55234.7,NaN,NaN,0.0
1995-09-11,9387,33,WITHDRAWAL,credit card withdrawal,4100.0,73559.4,NaN,NaN,0.0
1995-11-01,9390,33,WITHDRAWAL,credit card withdrawal,3300.0,57545.6,NaN,NaN,0.0


## Aggregation; Summary Statistics for Groups
When we use the aggregating functions on a GroupBy object, they are applied to all the columns by default. The resulting output is a DataFrame with the group names as the index.

### Example 1; Single Aggregating Function on Multiple Columns

In [53]:
# Only applies mean on the numeric variables.
grouped.mean()

trans_id   account_id  \
type       operation                                                 
CREDIT     Unknown                       3.568682e+06  3018.206303   
           collection from another bank  7.532214e+05  2551.594472   
           credit in cash                8.999767e+05  3042.470637   
NOT SURE   withdrawal in cash            1.231199e+06  4139.677187   
WITHDRAWAL credit card withdrawal        7.760621e+05  2627.217770   
           remittance to another bank    8.306999e+05  2808.375235   
           withdrawal in cash            8.776258e+05  2943.772056   

                                               amount       balance  \
type       operation                                                  
CREDIT     Unknown                         150.019603  36123.836008   
           collection from another bank  11981.111106  35636.781622   
           credit in cash                15429.855126  50108.124799   
NOT SURE   withdrawal in cash            12516.726149  45289.716027   
WITHDRAWAL credit card withdrawal         2261.124938  53271.422013   
           remittance to another bank     3229.441607  35413.966612   
           withdrawal in cash             5094.797375  36665.325692   

                                              account  
type       operation                                   
CREDIT     Unknown                                NaN  
           collection from another bank  4.995732e+07  
           credit in cash                         NaN  
NOT SURE   withdrawal in cash                     NaN  
WITHDRAWAL credit card withdrawal        0.000000e+00  
           remittance to another bank    4.912653e+07  
           withdrawal in cash            0.000000e+00

And obviously, it doesnt' make sense to apply the mean on 'trans_id', 'account_id', 'account'.

So let's filter for specific variables to apply the mean aggregating function.

In [54]:
grouped[['amount', 'balance']].mean()

amount       balance
type       operation                                               
CREDIT     Unknown                         150.019603  36123.836008
           collection from another bank  11981.111106  35636.781622
           credit in cash                15429.855126  50108.124799
NOT SURE   withdrawal in cash            12516.726149  45289.716027
WITHDRAWAL credit card withdrawal         2261.124938  53271.422013
           remittance to another bank     3229.441607  35413.966612
           withdrawal in cash             5094.797375  36665.325692

The 'describe' function generate basic descriptive statistics such as ;

    - count
    - mean
    - standard deviation
    - minimum
    - maximum
    - the main quartiles

In [55]:
# describe displays a bunch of statistics.
grouped[['amount', 'balance']].describe()

amount                              \
                                            count          mean           std   
type       operation                                                            
CREDIT     Unknown                       183114.0    150.019603     80.510640   
           collection from another bank   65226.0  11981.111106  14143.722583   
           credit in cash                156743.0  15429.855126  12041.062641   
NOT SURE   withdrawal in cash             16666.0  12516.726149   6593.286488   
WITHDRAWAL credit card withdrawal          8036.0   2261.124938   1213.570613   
           remittance to another bank    208283.0   3229.441607   2719.017605   
           withdrawal in cash            418252.0   5094.797375   8807.612329   

                                                                           \
                                            min     25%      50%      75%   
type       operation                                                        
CREDIT     Unknown                          0.0    89.7    133.0    195.6   
           collection from another bank  2900.0  3849.0   5550.0  12710.0   
           credit in cash                 100.0  5588.0  13564.0  22184.0   
NOT SURE   withdrawal in cash            2001.0  6738.5  12103.0  18081.0   
WITHDRAWAL credit card withdrawal         100.0  1300.0   2200.0   3000.0   
           remittance to another bank       1.0  1152.0   2536.0   4588.5   
           withdrawal in cash               0.0    14.6   1440.0   6000.0   

                                                   balance                \
                                             max     count          mean   
type       operation                                                       
CREDIT     Unknown                         565.5  183114.0  36123.836008   
           collection from another bank  74812.0   65226.0  35636.781622   
           credit in cash                49999.0  156743.0  50108.124799   
NOT SURE   withdrawal in cash            24997.0   16666.0  45289.716027   
WITHDRAWAL credit card withdrawal         8000.0    8036.0  53271.422013   
           remittance to another bank    14882.0  208283.0  35413.966612   
           withdrawal in cash            87400.0  418252.0  36665.325692   

                                                                           \
                                                  std      min        25%   
type       operation                                                        
CREDIT     Unknown                       19991.907822 -16390.2  21477.725   
           collection from another bank  24120.739918 -31178.7  20144.125   
           credit in cash                28315.646298 -25568.6  29326.150   
NOT SURE   withdrawal in cash            22477.989628 -10955.2  28802.600   
WITHDRAWAL credit card withdrawal        20170.730110 -11711.7  38813.425   
           remittance to another bank    19505.474108 -17030.4  20599.100   
           withdrawal in cash            19583.787291 -41125.7  22394.425   

                                                                        
                                              50%        75%       max  
type       operation                                                    
CREDIT     Unknown                       31319.85  46002.650  158959.2  
           collection from another bank  26792.75  42112.600  165654.3  
           credit in cash                45465.50  65488.450  209637.0  
NOT SURE   withdrawal in cash            41893.85  59784.725  174283.3  
WITHDRAWAL credit card withdrawal        51031.60  66046.175  155592.5  
           remittance to another bank    30387.40  46180.400  155219.5  
           withdrawal in cash            32442.70  46576.300  166743.2

The 'nunique' function displays the distinctive count for the groups.

In [56]:
grouped[['amount', 'balance']].min()

# only applies median on the numeric variables.
grouped[['amount', 'balance']].median()

grouped[['amount', 'balance']].max()

# The number of transactions in each group
grouped['trans_id'].count()

# The unique transactions and accounts in each group.
grouped[['trans_id', 'account_id']].nunique()

trans_id  account_id
type       operation                                         
CREDIT     Unknown                         183114        4453
           collection from another bank     65226        1606
           credit in cash                  156743        4500
NOT SURE   withdrawal in cash               16666        1144
WITHDRAWAL credit card withdrawal            8036         807
           remittance to another bank      208283        3602
           withdrawal in cash              418252        4500

The nunique function's results is above. We can see that trans_id is unique, while one account_id could have multiple transactions.

__Realted article__ ; The above are the most used groupby functions. You may also check out the 'https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats' for a full list.

### Example 2; Multiple Aggregating Functions on Multiple Columns
So far,we've been only applying a single aggregating function at a time. What if we want to apply multiple functions at once?

We can use the agg method to call multiple functions.

In [57]:
grouped[['amount', 'balance']].agg(['sum', 'mean', 'count'])

amount                        \
                                                  sum          mean   count   
type       operation                                                          
CREDIT     Unknown                       2.747069e+07    150.019603  183114   
           collection from another bank  7.814800e+08  11981.111106   65226   
           credit in cash                2.418522e+09  15429.855126  156743   
NOT SURE   withdrawal in cash            2.086038e+08  12516.726149   16666   
WITHDRAWAL credit card withdrawal        1.817040e+07   2261.124938    8036   
           remittance to another bank    6.726378e+08   3229.441607  208283   
           withdrawal in cash            2.130909e+09   5094.797375  418252   

                                              balance                        
                                                  sum          mean   count  
type       operation                                                         
CREDIT     Unknown                       6.614780e+09  36123.836008  183114  
           collection from another bank  2.324445e+09  35636.781622   65226  
           credit in cash                7.854098e+09  50108.124799  156743  
NOT SURE   withdrawal in cash            7.547984e+08  45289.716027   16666  
WITHDRAWAL credit card withdrawal        4.280891e+08  53271.422013    8036  
           remittance to another bank    7.376127e+09  35413.966612  208283  
           withdrawal in cash            1.533535e+10  36665.325692  418252

### Example 3 ; Custom Defined Function on Multiple Columns

In [59]:
import numpy as np

In [60]:
def sqrt_sum(x):
    return np.sqrt(x.sum())

grouped[['amount', 'balance']].agg([sqrt_sum])

amount        balance
                                             sqrt_sum       sqrt_sum
type       operation                                                
CREDIT     Unknown                        5241.248849   81331.298445
           collection from another bank  27954.962940   48212.495456
           credit in cash                49178.468683   88623.347970
NOT SURE   withdrawal in cash            14443.121477   27473.594728
WITHDRAWAL credit card withdrawal         4262.675216   20690.315302
           remittance to another bank    25935.261447   85884.382793
           withdrawal in cash            46161.771971  123835.963280

### Example 4 ; Custom Output Name on Multiple Columns
What if we want to change the output column names?

We can use the named aggregation syntax.

A dicitionary is created with;

    - the keys being the name of the output column
    - the values are tuples with;
        - the first element being the column to select and
        - the second element being the aggregating function to apply on that column
        
Then we use the ** operation to unpack the dictionary into the function.

It might sound a little confusing. Let's see an example below.

The 'aggs' dictionary defines the new output columns 'sum_amount', 'avg_balance', and 'avg_amount'.

For example, sum_amount is the new column name where the sum function is applied to the amount column.

In [61]:
aggs = {'sum_amount': ('amount', 'sum'),
       'avg_balance': ('balance', 'mean'),
       'avg_amount': ('amount', 'mean')}

grouped.agg(**aggs)

sum_amount   avg_balance  \
type       operation                                                  
CREDIT     Unknown                       2.747069e+07  36123.836008   
           collection from another bank  7.814800e+08  35636.781622   
           credit in cash                2.418522e+09  50108.124799   
NOT SURE   withdrawal in cash            2.086038e+08  45289.716027   
WITHDRAWAL credit card withdrawal        1.817040e+07  53271.422013   
           remittance to another bank    6.726378e+08  35413.966612   
           withdrawal in cash            2.130909e+09  36665.325692   

                                           avg_amount  
type       operation                                   
CREDIT     Unknown                         150.019603  
           collection from another bank  11981.111106  
           credit in cash                15429.855126  
NOT SURE   withdrawal in cash            12516.726149  
WITHDRAWAL credit card withdrawal         2261.124938  
           remittance to another bank     3229.441607  
           withdrawal in cash             5094.797375

## Transformation ; Like-indexed Object with Statistics for Groups

In this section, we'll look at the transform method.

It also allows us to apply functions to each group. What's the difference between transform and aggregation?

The transform method returns a result as an object that is indexed the same as the one being grouped.

The first argument to the transform function must be a series ( the group ) . 

It must either return a series with the same number of rows as the group chunk or broadcastable to the size of ther group chunk.

Usuall,y the broadcastable part just means a single scalar value.



### Example 1 ; Custom Defined Function on Multiple Columns - Range

We first define a function 'max_minus_min, which returns a scalar value ; the range (max-min)'. AS you can imagine, this range value is the same for every member of the same group.

We can then use the transform method to apply this function on the variables amount and balance.

In [62]:
def max_minus_min(x):
    return x.max() - x.min()

df_range = grouped[['amount', 'balance']].transform(max_minus_min).rename(columns={'amount': 'amount_range', 'balance' : 'balance_range'})

In [63]:
df_range

,amount_range,balance_range
date,,
1995-03-24,49899.0,235205.6
1995-04-13,71912.0,196833.0
1995-04-23,49899.0,235205.6
1995-04-30,565.5,175349.4
1995-05-13,71912.0,196833.0
...,...,...
1998-12-02,87400.0,207868.9
1998-12-10,71912.0,196833.0
1998-12-25,87400.0,207868.9


Since 'df_range' has the same order of rows as df, we can add these two new range variables back to the original dataset for further analysis.

In [66]:
df['amount_range'] = df_range['amount_range']
df['balance_range'] = df_range['balance_range']

you can print out the head of df to see how the amount_range and balance_range are the same for rows of the same groups. 

### Example 2 ; Custom Defined Function on Multiple Columns - Percentage

Another common example would be getting the percentage of the value among its group.

We define the 'x_pct' function with input x as a series. And it returns the percentage series the same size as x.

Then we use the transform method to pass on this new function to variables amount and balance.

In [67]:
def x_pct(x):
    return x/x.sum()

df_pct = grouped[['amount', 'balance']].transform(x_pct).rename(columns={'amount': 'amount_pct', 'balance': 'balance_pct'})
df_pct

,amount_pct,balance_pct
date,,
1995-03-24,4.134757e-07,1.273221e-07
1995-04-13,4.707734e-06,2.012954e-06
1995-04-23,5.209794e-06,2.199998e-06
1995-04-30,6.989268e-07,2.615083e-06
1995-05-13,4.707734e-06,9.024607e-06
...,...,...
1998-12-02,1.201365e-05,2.681022e-06
1998-12-10,5.918002e-05,3.758420e-05
1998-12-25,2.956485e-06,5.285984e-06


### Example 3 ; Custom Defined Function on Multiple COlumns - TimeSeires

The most common usage of transform for us is creating time series features. It is handy when we need to use a rolling window to calculate things that happended in a previous time frame.

#### Plz note that pandas does have a rolling function

But when we need to apply the function to groups, the best way is to use 'groupby_s' transform method.

Since the datset is for bank transactions. We often want to see the volume transacted within the past 'n_days',  for each paricular 'account ID'.

These types of features are useful for customer behavior analysis, such as fraud analytics, anti-money laundering detection.



The Python code below does the following steps to create the rolling variables;

    - create a GroupBy object by account_id since we want to look at account level activiteds
    - define the function 'sum_n_days' with input x ( a series ) and n ( integer ). The x is a times series on which we can calculate the sum of its value, within the last 'n_days'
    - use the transform method to apply this function on the amount and balance variables
    - add back these two new variables to the original datset

In [70]:
groupby_account = df.groupby('account_id')

def sum_n_days(x, n):
    # define the timedelta
    window_length = timedelta(days=n) # timedelta?????
    
    # x is a series. Use the series rolling function.
    return x.rolling(window_length).sum()

df_sum = groupby_account[['amount', 'balance']].transform(sum_n_days, n=5).rename(columns={'amount': 'amount_5days', 'balance': 'balance_5days'})

# add to the original datset.
df['amount_5days'] = df_sum['amount_5days']
df['balance_5days'] = df_sum['balance_5days']

# check an example
df[df['account_id'] == 2].head(10)

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,amount_range,balance_range,amount_5days,balance_5days
date,,,,,,,,,,,,,
1993-02-26,276,2,CREDIT,credit in cash,1100.0,1100.0,NaN,NaN,NaN,49899.0,235205.6,1100.0,1100.0
1993-03-12,279,2,CREDIT,collection from another bank,20236.0,21336.0,NaN,ST,66487163.0,71912.0,196833.0,20236.0,21336.0
1993-03-28,697,2,CREDIT,credit in cash,3700.0,25036.0,NaN,NaN,NaN,49899.0,235205.6,3700.0,25036.0
1993-03-31,3530483,2,CREDIT,Unknown,13.5,25049.5,UROK,NaN,NaN,565.5,175349.4,3713.5,50085.5
1993-04-12,280,2,CREDIT,collection from another bank,20236.0,45285.5,NaN,ST,66487163.0,71912.0,196833.0,20236.0,45285.5
1993-04-27,698,2,WITHDRAWAL,withdrawal in cash,11000.0,34285.5,NaN,NaN,NaN,87400.0,207868.9,11000.0,34285.5
1993-04-30,3530484,2,CREDIT,Unknown,109.5,34394.9,UROK,NaN,NaN,565.5,175349.4,11109.5,68680.4
1993-05-12,281,2,CREDIT,collection from another bank,20236.0,54630.9,NaN,ST,66487163.0,71912.0,196833.0,20236.0,54630.9
1993-05-27,699,2,WITHDRAWAL,withdrawal in cash,17600.0,37030.9,NaN,NaN,NaN,87400.0,207868.9,17600.0,37030.9


As you can see from row 4 , there are two transactions within the past 5days (including the current one on 1993-03-01). So amount_5days is 3713.5(3700+13.5)

## Apply; Flexible Statistics for Groups but Less Efficient
Lastly, let's talk about the apply function, which is also a method to apply functions group-wise.

    Some operations on the grouped data might not fit into either the aggregate or transform categories. 
    Or, you may simply want GroupBy to infer how to combine the results.
    
    For these, use the 'apply' function, which can be substituted for both 'aggregate' and 'transform' in many standard use cases.
    However, 'apply' can handle some exceptional use cases.
    
    https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
    
For this reason, we haven't found much need to use it. It is still a popular function, so try it out yourself to decide whether to use it.

First, we also need to define a function that will be passed onto apply. The function must take a DataFrame as its first argument and return a DataFrame, series, or scalar.

'apply' will then take care of combining the results back together into a single dataframe or seires.

### Example 1; Multiple Columns Returning a DataFrame

    - amount_div_balance ; the percentage of amount out of balance of the account
    - amount_zscore ; the 'z score' for  the amount
    
As you can see, apply can have functions using multiple columns from the dataset ( amount and balance ).

Then we pass on the function f to apply on amount and balance.

In [ ]:
def f(group):
    return pd.DataFrame({'amount_div_balance': group['amount']/group['balance'],
                        'amount_zscore': (group['amount'] - group['amount'].mean())/group['amount'].std()})

groupby_account[['amount', 'balance']].apply(f)

Again, you can use the same method as in Example 1 of transform, to add these new variables to the original datset df for further analysis.

### Example 2; Multiple Columns Returning a Scalar

The function 'g' uses amount and balance to calculate their mean's comparison, which is a scalar value.

Then we can use apply again to apply the function 'g'.

In [ ]:
def g(group):
    return group['amount'].mean()/group['balance'].mean()

groupby_account[['amount', 'balance']].apply(g)